In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from nltk.tree import Tree
import svgling

import networkx as nx
from scipy import sparse

from IPython import embed

In [2]:
import pickle

save_dir = "D:\\MIT\\Courses\\2023-Spring\\6.8630-NLP\\Final Project\\LLM CGW\\All Parse Trees"

In [ ]:
with open(save_dir+'\\Trees\\'+'PCFG'+'_Base_Trees.pkl', 'rb') as f:
    trees = pickle.load(f)

print(len(trees[0]))
display(trees[0])

In [3]:
models = ['PCFG', 'ChatGPT', 'Google']

all_trees = {}
for model in models:
    temp = []
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
    temp.extend(trees)
    with open(save_dir+'\\Trees\\'+model+'_Adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
    temp.extend(trees)
    
    all_trees[model] = temp

### Rules-related Computation for Models

In [6]:
from nltk.tree.probabilistic import ProbabilisticTree

def NT_rule_counts(treebank):
    # The input is treebank, a list of NLTK trees
    # You will return 
    #  - NTs, a list of unique nonterminals
    #  - count_NT, a dictionary mapping NTs to their counts
    #  - count_rule, a dictionary mapping rules (represented as nested tuples ("S", ("NP", "VP")) ) to their counts
    
    NTs = []
    count_NT = {}
    count_rule = {}
    
    # Your implementation here
    queue = []
    for tree in treebank:
        queue.append(tree)

        # DFS Traverse for Each Tree
        while len(queue)>0:
            current_node = queue.pop()
            #print(current_node)
        
            lhs_label = current_node.label()
            rhs_labels = []
            for i in range(len(current_node)):
                if type(current_node[i]) in [ProbabilisticTree, Tree]:
                    rhs_labels.append(current_node[i].label())
                else:
                    rhs_labels.append(current_node[i])
            children = [current_node[i] for i in range(len(current_node))]

            if lhs_label not in NTs:
                NTs.append(lhs_label)
        
            if lhs_label not in list(count_NT.keys()):
                count_NT[lhs_label] = 1
            else:
                count_NT[lhs_label] += 1
        
            rule = (lhs_label, tuple(rhs_labels))
            if rule not in list(count_rule.keys()):
                count_rule[rule] = 1
            else:
                count_rule[rule] += 1
        
            for node in children:
                if type(node) in [ProbabilisticTree, Tree]:
                    queue.append(node)

    
    return NTs, count_NT, count_rule

In [7]:
NTs = {}
count_NTs = {}
count_rules = {}

models = ['PCFG', 'ChatGPT', 'Google']
for model in models:
    NTs[model], count_NTs[model], count_rules[model] = NT_rule_counts(all_trees[model])

## PCFG Overview

In [9]:
print(NTs['PCFG'])

['S1', 'End', 'S', 'VPSi', 'VPPastSi', 'NPObj', 'NPObjNoAppo', 'NPObjSi', 'NPSi', 'NPSiPostAdj', 'Loc', 'To', 'VTPast', 'NPSubSi', 'Proper', 'VPPresSi', 'NbarSi', 'NounSi', 'PreNPSi', 'DetSi', 'BePresSi', 'PP', 'Prep', 'VTPresSi', 'Adj', 'BeSi', 'VTGer', 'SQ', 'VPBaseQuesYN', 'VIBase', 'NPSubPl', 'NPPl', 'NPPlPostAdj', 'NounPl', 'DoPl', 'VPBaseQues', 'DoSi', 'WhPronoun', 'SQClause', 'VPQues', 'NPSub', 'NPSubNoAppo', 'ProNounSubSi', 'VTBase', 'VPBase', 'WhAdverb', 'VPGerQuesYN', 'SClause', 'SBase', 'ProNounSubPl', 'BePl', 'BePresPl', 'VPPl', 'VPPresPl', 'NbarPl', 'PreNPPl', 'NumPl', 'VPPastPart', 'VIPastPart', 'HaveBase', 'MDPast', 'VPGer', 'ObjPrep', 'BeBase', 'MD', 'NPSiClause', 'NPSiClausePres', 'RelClausePron', 'VIPresSi', 'ProNounObjSi', 'NPSubSiTer', 'VTPastPart', 'BePastSi', 'BePastPart', 'PosProNoun', 'CoordConj', 'VTPl', 'VTPresPl', 'SubConj', 'Adverb', 'VIPresPl', 'NPObjPl', 'NPSubSiAppo', 'NPObjWithAppo', 'NPObjSiAppo', 'NPSubPlTer', 'VTSi', 'NPClausePast', 'VPPast', 'Pause',

In [10]:
len(NTs['PCFG'])

104

In [11]:
print(count_NTs['PCFG'])

{'S1': 116, 'End': 80, 'S': 109, 'VPSi': 77, 'VPPastSi': 6, 'NPObj': 66, 'NPObjNoAppo': 68, 'NPObjSi': 72, 'NPSi': 175, 'NPSiPostAdj': 61, 'Loc': 24, 'To': 11, 'VTPast': 5, 'NPSubSi': 99, 'Proper': 42, 'VPPresSi': 40, 'NbarSi': 100, 'NounSi': 92, 'PreNPSi': 97, 'DetSi': 96, 'BePresSi': 10, 'PP': 20, 'Prep': 20, 'VTPresSi': 17, 'Adj': 7, 'BeSi': 5, 'VTGer': 10, 'SQ': 34, 'VPBaseQuesYN': 7, 'VIBase': 22, 'NPSubPl': 37, 'NPPl': 32, 'NPPlPostAdj': 22, 'NounPl': 30, 'DoPl': 10, 'VPBaseQues': 13, 'DoSi': 27, 'WhPronoun': 13, 'SQClause': 4, 'VPQues': 4, 'NPSub': 9, 'NPSubNoAppo': 15, 'ProNounSubSi': 5, 'VTBase': 13, 'VPBase': 18, 'WhAdverb': 7, 'VPGerQuesYN': 17, 'SClause': 9, 'SBase': 4, 'ProNounSubPl': 12, 'BePl': 14, 'BePresPl': 16, 'VPPl': 27, 'VPPresPl': 11, 'NbarPl': 8, 'PreNPPl': 8, 'NumPl': 7, 'VPPastPart': 5, 'VIPastPart': 2, 'HaveBase': 6, 'MDPast': 15, 'VPGer': 8, 'ObjPrep': 7, 'BeBase': 7, 'MD': 13, 'NPSiClause': 7, 'NPSiClausePres': 6, 'RelClausePron': 9, 'VIPresSi': 19, 'ProNoun

In [13]:
len(count_rules['PCFG'])

332

## ChatGPT Overview

In [14]:
print(NTs['ChatGPT'])

['S', '.', 'VP', 'PP', 'NP', 'NNP', 'TO', 'VBD', 'NN', 'DT', 'VBZ', 'IN', 'JJ', 'VBG', '?', 'VB', 'NNS', 'WP', 'PRP', 'WHADVP', 'WRB', 'VBP', 'CD', 'VBN', 'MD', '!', 'SBAR', 'WHNP', 'CC', 'PRP$', 'ADVP', 'RB', ',', 'WDT', 'Aux', 'ADP', 'ADJP', 'SBARQ', 'PUNCT', 'SQ']


In [15]:
len(NTs['ChatGPT'])

40

In [16]:
print(count_NTs['ChatGPT'])

{'S': 132, '.': 57, 'VP': 175, 'PP': 35, 'NP': 183, 'NNP': 76, 'TO': 16, 'VBD': 10, 'NN': 93, 'DT': 104, 'VBZ': 75, 'IN': 27, 'JJ': 9, 'VBG': 26, '?': 29, 'VB': 56, 'NNS': 29, 'WP': 17, 'PRP': 23, 'WHADVP': 4, 'WRB': 6, 'VBP': 25, 'CD': 9, 'VBN': 10, 'MD': 19, '!': 22, 'SBAR': 15, 'WHNP': 15, 'CC': 26, 'PRP$': 3, 'ADVP': 3, 'RB': 4, ',': 3, 'WDT': 5, 'Aux': 4, 'ADP': 1, 'ADJP': 2, 'SBARQ': 1, 'PUNCT': 8, 'SQ': 1}


In [17]:
len(count_rules['ChatGPT'])

348

## Google Overview

In [18]:
print(NTs['Google'])

['root', 'p', 'PUNCT', 'prep', 'pobj', 'NOUN', 'ADP', 'VERB', 'nsubj', 'attr', 'det', 'DET', 'dobj', 'acomp', 'ADJ', 'csubj', 'aux', 'PRON', 'ccomp', 'advmod', 'ADV', 'nn', 'num', 'NUM', 'nsubjpass', 'rcmod', 'title', 'conj', 'cc', 'CONJ', 'preconj', 'amod', 'auxpass', 'poss', 'advcl', 'mark', 'neg', 'appos', 'dep', 'vmod', 'pcomp', 'xcomp', 'PRT']


In [19]:
len(NTs['Google'])

43

In [20]:
print(count_NTs['Google'])

{'root': 232, 'p': 120, 'PUNCT': 120, 'prep': 76, 'pobj': 61, 'NOUN': 207, 'ADP': 41, 'VERB': 220, 'nsubj': 199, 'attr': 30, 'det': 87, 'DET': 98, 'dobj': 50, 'acomp': 4, 'ADJ': 7, 'csubj': 6, 'aux': 61, 'PRON': 50, 'ccomp': 20, 'advmod': 11, 'ADV': 12, 'nn': 13, 'num': 7, 'NUM': 8, 'nsubjpass': 9, 'rcmod': 16, 'title': 4, 'conj': 31, 'cc': 17, 'CONJ': 31, 'preconj': 13, 'amod': 3, 'auxpass': 3, 'poss': 2, 'advcl': 2, 'mark': 2, 'neg': 12, 'appos': 4, 'dep': 5, 'vmod': 12, 'pcomp': 1, 'xcomp': 4, 'PRT': 2}


In [21]:
len(count_rules['Google'])

320